In [19]:
import os, sys
sys.path.insert(0, os.path.abspath('..'))
import numpy as np
import nashpy as nash
from IPython.display import display
import support_enumeration as se
import BNE as BNE
import IESDS as IESDS

### 1.1

In [20]:
for x in [1, 4, 10]:
    U1 = np.array([[4 - x, 10 - x], [0, 0]])
    U2 = np.array([[4 - x, 0], [10 - x, 0]])

    NE = se.msne(U1, U2)

    print(NE)

[(array([1., 0.]), array([1., 0.]))]
[(array([1., 0.]), array([1., 0.])), (array([1., 0.]), array([0., 1.])), (array([0., 1.]), array([1., 0.]))]
[(array([1., 0.]), array([0., 1.])), (array([0., 1.]), array([1., 0.])), (array([0., 1.]), array([0., 1.]))]


### 1.2

In [21]:
for x in [1, 4, 100]:
    U1 = np.array([[4 - x, 4 - x], [10 - x, 0]])
    U2 = np.array([[0, 10 - x], [0, 0]])

    NE = se.msne(U1, U2)

    print(NE)

[(array([1., 0.]), array([0., 1.])), (array([0., 1.]), array([1., 0.]))]
[(array([1., 0.]), array([0., 1.])), (array([0., 1.]), array([1., 0.])), (array([0., 1.]), array([0., 1.]))]
[(array([0., 1.]), array([1., 0.])), (array([0., 1.]), array([0., 1.]))]


c:\Users\elias\anaconda3\Lib\site-packages\nashpy\algorithms\support_enumeration.py:260: RuntimeWarning: 
An even number of (2) equilibria was returned. This
indicates that the game is degenerate. Consider using another algorithm
to investigate.
                  
  warnings.warn(warning, RuntimeWarning)


### 1.3

Lavet i lyx

### 1.4

In [22]:
UA_5 = np.array([[-1, 5], 
                [0, 0]])
# U2A = np.array([[3, 4], 
#                 [0, -1]])


UB_3 = np.array([[1, 0], 
                [7, 0]])
UB_7 = np.array([[-3, 0], 
                [3, 0]])


### (a)

In [23]:
NE1 = se.msne(UA_5, UB_3)

print(NE1)

NE2 = se.msne(UA_5, UB_7)
print(NE2)


[(array([0., 1.]), array([1., 0.]))]
[(array([1., 0.]), array([0., 1.])), (array([0., 1.]), array([1., 0.])), (array([0.5, 0.5]), array([0.83333333, 0.16666667]))]


### (b)

In [24]:
U1 = [UA_5 ,UA_5]
U2 = [UB_3,UB_7]

action = ['I','U']

BU1, BU2 = BNE.compute_full_matrix(U1,U2,(1/2), action, action)

#IESDS
U1, U2, aa1, aa2 = IESDS.IESDS(BU1[0], BU2[0])

result = BNE.compute_bne(U1,U2)

print(f"p = {1/2}: BNE {result}")
display(BU1[1])
display(BU2[1])

a=2 is strictly dominated by a=0
a=3 is strictly dominated by a=1
No further strategies to delete after 1 iterations
p = 0.5: BNE [(array([1., 0.]), array([0., 1.])), (array([0., 1.]), array([1., 0.])), (array([0.5, 0.5]), array([0.66666667, 0.33333333]))]


,II,IU,UI,UU
I,-1.0,2.0,2.0,5.0
U,0.0,0.0,0.0,0.0


,II,IU,UI,UU
I,-1.0,0.5,-1.5,0.0
U,5.0,3.5,1.5,0.0
